In [6]:
class CharField:
    def __init__(self, min_=None, max_=None):
        min_ = min_ or 0  # in case min_ is None
        min_ = max(min_, 0)  # replaces negative value with zero
        self._min = min_
        self._max = max_

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.prop_name} must be a string.')
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min} chars.')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max} chars')
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)
    

# 01 - Enumerations

#### Motivation

How do we deal with collections of related constants? One way would be:

In [2]:
RED = 1
GREEN = 2
BLUE = 3

COLOURS = (RED, GREEN, BLUE)

RED in COLOURS

True

But this causes the following issues:

In [5]:
print(1 in COLOURS)
print(RED < GREEN)

True
True


Even if we were to assign the symbols to meaningful strings e.g. `RED = 'red'`, we can run into issues and accidents such as having non-unique values by doing `GREEN = 'red'` for example. We could also do `RED * 2`...

What do we really want? 

An **immutable** collection of related **constant** members:
- having unique names e.g. RED
- having an associated **constant** value e.g. 1
- having unique associated values
- not allowing operations e.g. `RED * 2`
- lookup member by name
- lookup member by value


**Aliases**

Sometimes we want multiple symbols to refer to the same thing:
```python
POLY_4 = 4
RECTANCLE = 4
SQUARE = 4
RHOMBUS = 4
```
Here, the last three are aliases for `POLY_4`. With reverse lookups, we want to return the original, not the alias, so `4 -> POLY_4`.

#### Lecture

Here's the basic setup:

In [12]:
from enum import Enum

class Colour(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

Here's some terminology:

- `Colour` is called an **enumeration** (enum for short)
- `Colour.RED` is called an **enumeration member** (and their values are just called values)
- The `type` of a member is the **enumeration** that it belongs to. This is odd; see below (and metaprogramming sections)

In [22]:
print(type(Colour.RED))

<enum 'Colour'>


In [17]:
class Colour2:
    RED = 1

type(Colour2.RED)

int

Here's some basics:

In [25]:
print(f"{isinstance(Colour.RED, Colour) = }")
print(f"{str(Colour.RED) = }")
print(f"{repr(Colour.RED) = }")
print(f"{Colour.RED.name = }")
print(f"{Colour.RED.value = }")


isinstance(Colour.RED, Colour) = True
str(Colour.RED) = 'Colour.RED'
repr(Colour.RED) = '<Colour.RED: 1>'
Colour.RED.name = 'RED'
Colour.RED.value = 1


**Equality and Membership**

Equality is done using identity:`is` (but `==` works too, but shouldn't be used as it can be overridden)

In [26]:
Colour.GREEN in Colour

True

But note that members and their associated values are **not equal**!

In [29]:
Colour.RED == 1

False

**Hashability**

Enumeration members are always **hashable**. This is so that they can be used as:
- keys in dictionaries
- elements of a set

For example:

In [30]:
pixel_colour = {
    Colour.RED: 100,
    Colour.GREEN: 25,
    Colour.BLUE: 255
}

**Lookup**

There are two ways we can retrieve a member. Either through its value directly (e.g. 1 for `Colour.RED`) or by the string of the member name (e.g. `'RED'`).

To do this, know that enumerations are **callables**:

In [32]:
Colour(1)

<Colour.RED: 1>

In [33]:
Colour['RED']

<Colour.RED: 1>

**Enumerating Members**

Enumerations are **iterables**, so we can `list(Colour)` with **definition order** preserved:

In [34]:
class Colour(Enum):
    GREEN = 2
    RED = 1
    BLUE = 3

list(Colour)

[<Colour.GREEN: 2>, <Colour.RED: 1>, <Colour.BLUE: 3>]

We also have the `__members__` property which returns a dict where the keys are the names as strings, and the values are the values.

In [37]:
Colour.__members__

mappingproxy({'GREEN': <Colour.GREEN: 2>,
              'RED': <Colour.RED: 1>,
              'BLUE': <Colour.BLUE: 3>})

**Constant Members**

Once an enumeration has been declared:
- member list is immutable (cannot add or remove members)
- member values are immutable
- cannot be subclassed, **unless it contains no members**. You'll see why creating empty enumerations can be useful.

# 02 - Aliases

# 03 - Customizing and Extending Enumerations

# 04 - Automatic Values